In [ ]:
#First let us do the X-basis encoding   

In [2]:
#Importing
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from random import randrange

In [3]:
def print_outcomes_in_reserve(counts):
    for outcome in counts:
        reverse_outcome = ''
        for i in outcome:
            reverse_outcome = i+ reverse_outcome
    return reverse_outcome



def SendState(qc1,qc2,qc1_name):
    """
    This function takes one circuits (qc1s) output
    and creates initializes another circuit qc2 with the same state
    """
    #Here quantum state retrieved from the qasm code of qc1
    qs = qc1.qasm().split(sep = ';')[4:-1]

    #process the code to get the instructions
    for index, instruction in enumerate(qs):
        qs[index] = instruction.lstrip()

    for instruction in qs:
        if instruction[0] == 'x':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])

            qc2.x(qreg[old_qr])
        elif instruction[0] == 'h':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])

            qc2.h(qreg[old_qr])
        elif instruction[0] == 'm': #We are excluding the measuring
            pass
        else:
            raise Exception('Unable to parse instruction')

In [4]:
qreg = QuantumRegister(8)
creg = ClassicalRegister(8)

#Creating Quantum Circuit for Asja State

asja  = QuantumCircuit(qreg,creg, name = 'Asja')

send = []


#Creating random bit string

for i in range(8):
    bit = randrange(2)
    send.append(bit)


#Applying X gate if bit is equal to 1
    
for i,n in enumerate(send):
    if n==1:
        asja.x(qreg[i])

send_str = ''.join(str(e) for e in send)

In [ ]:
#Now let us switch to the X basis by applying the H gates

In [5]:
asja.h(qreg)

asja.draw()

┌───┐┌───┐
q0_0: ┤ X ├┤ H ├
      ├───┤├───┤
q0_1: ┤ X ├┤ H ├
      ├───┤├───┤
q0_2: ┤ X ├┤ H ├
      ├───┤└───┘
q0_3: ┤ H ├─────
      ├───┤     
q0_4: ┤ H ├─────
      ├───┤     
q0_5: ┤ H ├─────
      ├───┤┌───┐
q0_6: ┤ X ├┤ H ├
      ├───┤├───┤
q0_7: ┤ X ├┤ H ├
      └───┘└───┘
c0: 8/══════════

In [6]:
balvis = QuantumCircuit(qreg, creg, name='Balvis') #Defining Balvis circuit

SendState(asja, balvis, 'Asja') #Asja sends states to Balvis

balvis.draw()

C:\Users\jrnat\AppData\Local\Temp\ipykernel_30496\1016264951.py:16: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.qasm()`` is deprecated as of qiskit 0.46. It will be removed in the 1.0.0 release. Instead use the qiskit.qasm2.dump() or qiskit.qasm2.dumps() function
  qs = qc1.qasm().split(sep = ';')[4:-1]


┌───┐┌───┐
q0_0: ┤ X ├┤ H ├
      ├───┤├───┤
q0_1: ┤ X ├┤ H ├
      ├───┤├───┤
q0_2: ┤ X ├┤ H ├
      ├───┤└───┘
q0_3: ┤ H ├─────
      ├───┤     
q0_4: ┤ H ├─────
      ├───┤     
q0_5: ┤ H ├─────
      ├───┤┌───┐
q0_6: ┤ X ├┤ H ├
      ├───┤├───┤
q0_7: ┤ X ├┤ H ├
      └───┘└───┘
c0: 8/══════════

In [7]:
balvis.h(qreg) #Applying H gate first
balvis.measure(qreg,creg) #then continue with regular measurement

job = execute(balvis,Aer.get_backend('qasm_simulator'),shots=1) #Note that Balvis only has one shot to measure qubits
counts = job.result().get_counts(balvis) # counts is a dictionary object in python
received = print_outcomes_in_reserve(counts)

print("Asja sent:", send_str)
print("Balvis received:", received)

C:\Users\jrnat\AppData\Local\Temp\ipykernel_30496\507994298.py:4: DeprecationWarning: The 'qiskit.Aer' entry point is deprecated and will be removed in Qiskit 1.0. You should use 'qiskit_aer.Aer' directly instead.
  job = execute(balvis,Aer.get_backend('qasm_simulator'),shots=1) #Note that Balvis only has one shot to measure qubits
C:\Users\jrnat\AppData\Local\Temp\ipykernel_30496\507994298.py:4: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(balvis,Aer.get_backend('qasm_simulator'),shots=1) #Note that Balvis only has one shot to measure qubits


Asja sent: 11100011
Balvis received: 11100011


In [ ]:
#Now let us do something fun
#We will have 16 bits, 8 bits encoded in X basis and next 8 bits in Z basis but
#Balvis measures them in opposite order of Z and X basis

In [10]:
# import all necessary objects and methods for quantum circuits
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from random import randrange

def print_outcomes_in_reserve(counts): # takes a dictionary variable
    for outcome in counts: # for each key-value in dictionary
        reverse_outcome = ''
        for i in outcome: # each string can be considered as a list of characters
            reverse_outcome = i + reverse_outcome # each new symbol comes before the old symbol(s)
    return reverse_outcome

#Source for SendState: awards/teach_me_qiskit_2018/cryptography/Cryptography.ipynb

def SendState(qc1, qc2, qc1_name):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates and initializes another circuit qc2 with the same state
    ''' 
    
    # Quantum state is retrieved from qasm code of qc1
    qs = qc1.qasm().split(sep=';')[4:-1]

    # Process the code to get the instructions
    for index, instruction in enumerate(qs):
        qs[index] = instruction.lstrip()

    # Parse the instructions and apply to new circuit
    for instruction in qs:
        if instruction[0] == 'x':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.x(qreg[old_qr])
        elif instruction[0] == 'h':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.h(qreg[old_qr])
        elif instruction[0] == 'm': # exclude measuring:
            pass
        else:
            raise Exception('Unable to parse instruction')

qreg = QuantumRegister(16) # quantum register with 8 qubits
creg = ClassicalRegister(16) # classical register with 8 bits

# Quantum circuit for Asja state
asja = QuantumCircuit(qreg, creg, name='Asja')

send=[] #Initial bit string to send

#Creating random bit string
for i in range(16):
    bit = randrange(2)
    send.append(bit)
    
#Apply X gate if bit is equal to 1
for i, n in enumerate(send):
    if n==1:
        asja.x(qreg[i]) # apply x-gate
#        print("x-gate is applied to the qubit with index",i)

send_str = ''.join(str(e) for e in send)

for i in range(8):
    asja.h(qreg[i]) #first 8 qubits encoded in X basis

asja.draw() #To check gates applied correctly

┌───┐     
 q5_0: ┤ H ├─────
       ├───┤┌───┐
 q5_1: ┤ X ├┤ H ├
       ├───┤└───┘
 q5_2: ┤ H ├─────
       ├───┤     
 q5_3: ┤ H ├─────
       ├───┤┌───┐
 q5_4: ┤ X ├┤ H ├
       ├───┤└───┘
 q5_5: ┤ H ├─────
       ├───┤     
 q5_6: ┤ H ├─────
       ├───┤     
 q5_7: ┤ H ├─────
       ├───┤     
 q5_8: ┤ X ├─────
       ├───┤     
 q5_9: ┤ X ├─────
       ├───┤     
q5_10: ┤ X ├─────
       ├───┤     
q5_11: ┤ X ├─────
       └───┘     
q5_12: ──────────
       ┌───┐     
q5_13: ┤ X ├─────
       └───┘     
q5_14: ──────────
       ┌───┐     
q5_15: ┤ X ├─────
       └───┘     
c2: 16/══════════

In [11]:
balvis = QuantumCircuit(qreg, creg, name='Balvis') #Defining Balvis circuit
SendState(asja, balvis, 'Asja') #Asja sends states to Balvis

# Balvis measures first 8 qubits using Z-basis (0 - 7) and next 8 qubits using X-basis (8 - 15)
for n in range(8, 16):
    balvis.h(qreg[n])
    
balvis.draw()

C:\Users\jrnat\AppData\Local\Temp\ipykernel_30496\1019178779.py:20: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.qasm()`` is deprecated as of qiskit 0.46. It will be removed in the 1.0.0 release. Instead use the qiskit.qasm2.dump() or qiskit.qasm2.dumps() function
  qs = qc1.qasm().split(sep=';')[4:-1]


┌───┐     
 q5_0: ┤ H ├─────
       ├───┤┌───┐
 q5_1: ┤ X ├┤ H ├
       ├───┤└───┘
 q5_2: ┤ H ├─────
       ├───┤     
 q5_3: ┤ H ├─────
       ├───┤┌───┐
 q5_4: ┤ X ├┤ H ├
       ├───┤└───┘
 q5_5: ┤ H ├─────
       ├───┤     
 q5_6: ┤ H ├─────
       ├───┤     
 q5_7: ┤ H ├─────
       ├───┤┌───┐
 q5_8: ┤ X ├┤ H ├
       ├───┤├───┤
 q5_9: ┤ X ├┤ H ├
       ├───┤├───┤
q5_10: ┤ X ├┤ H ├
       ├───┤├───┤
q5_11: ┤ X ├┤ H ├
       ├───┤└───┘
q5_12: ┤ H ├─────
       ├───┤┌───┐
q5_13: ┤ X ├┤ H ├
       ├───┤└───┘
q5_14: ┤ H ├─────
       ├───┤┌───┐
q5_15: ┤ X ├┤ H ├
       └───┘└───┘
c2: 16/══════════

In [12]:
balvis.measure(qreg,creg)

job = execute(balvis,Aer.get_backend('qasm_simulator'),shots=1) #Note that Balvis only has one shot to measure qubits
counts = job.result().get_counts(balvis) # counts is a dictionary object in python
received = print_outcomes_in_reserve(counts)

print("Asja sent:", send_str)
print("Balvis received:", received)

Asja sent: 0100100011110101
Balvis received: 0101000000111011


C:\Users\jrnat\AppData\Local\Temp\ipykernel_30496\2855181941.py:3: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(balvis,Aer.get_backend('qasm_simulator'),shots=1) #Note that Balvis only has one shot to measure qubits
